In [1]:
import numpy as np
import cv2
import glob
import math

In [134]:
# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :

	# open the actual image in color mode
	img1 = cv2.imread ( image , cv2.IMREAD_COLOR )
	img13 = cv2.imread ( image , cv2.IMREAD_COLOR )

	## mask o yellow (15,0,0) ~ (36, 255, 255)

	# blue 0 100
	# green 0 255
	# red 0 255

	# dimension of the object plus a small piece of the glass
	img = img1 [ int ( 1080 / 8 * 1 ) :int ( 1080 / 8 * 7 ) , int ( 1920 / 6 ) :int ( 1920 / 2 ) , : ]
	img [ 0 :int ( 1080 / 8 * 6 ) , 0 :int ( 1920 / 3 ) , : ] = img
	mask2 = cv2.inRange ( img , (55 , 35 , 0) , (170 , 120 , 98) )
	target = cv2.bitwise_and ( img , img , mask = mask2 )

	#yellow glass filter
	img_w = img13 [ int ( 1080 / 8 * 1 ) :int ( 1080 / 8 * 7 ) , int ( 1920 / 2-(1920/18)) :int ( 1920 / 2 ) , : ]
	mask3 = cv2.inRange ( img_w , (0,150,150) , (130,255,230) )
	target1 = cv2.bitwise_and ( img_w , img_w , mask = mask3 )


	#kernel = np.ones ( (10,10) , np.uint8 )
	#target = cv2.erode ( target , kernel )

	yellow_filter = image.replace ( "uncalibrated" , "backgroundremoved" )

	cv2.imwrite ( yellow_filter , target1 )